In [84]:
import pandas as pd
import os
from utils import drop_categorical_columns
from utils import missing_values_table
from sklearn.model_selection import train_test_split
import numpy as np

In [85]:


def get_Xy(data, label):
    X = data.drop(label, axis=1)
    y = data[label]
    return X, y

In [86]:
num=1
if num<2:
    df = pd.read_csv("Final-Datasets/UCI-Seacom-Encoded.csv")
    label = "Pass/Fail"
    random_seed = np.random.randint(1, 10000)
    X, y = get_Xy(
        df, "Pass/Fail"
    )  # Features (all columns except the last one)  # Labels (the last column)

    # Split the data into training and testing sets (adjust the test_size as needed)
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_seed
    )
    num=Y_train.nunique()
    if num==2:
        print(random_seed)


6296


In [87]:
def drop_label_with_null(df, column_name):
    # Drop rows where the specified column is null
    df_cleaned = df.dropna(subset=[column_name])

    return df_cleaned

In [88]:
path = os.path.join('Final-Datasets', 'uci-secom.csv')
original = pd.read_csv(path)
original.columns


Index(['Time', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '581', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=592)

In [89]:
dropped=drop_label_with_null(original, 'Pass/Fail')
dropped.shape

(1567, 592)

In [90]:
dropped.head(3)

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1


In [91]:
dropped['Time'][0]

'2008-07-19 11:55:00'

In [92]:
dropped['Time'] = pd.to_datetime(dropped['Time']).astype(int)
dropped.drop(columns=['292','293','157','158','250','220','85','492','358'],inplace=True)

In [93]:
dropped['Pass/Fail'].nunique()

2

In [94]:
categorical_columns = dropped.select_dtypes(include=['object', 'category']).columns.tolist()
print(len(categorical_columns))
for col in categorical_columns:
    dropped[col] = pd.to_numeric(dropped[col], errors='ignore')
# after those are taken care of we can drop the columns that are still object
categorical_columns = dropped.select_dtypes(include=['object', 'category']).columns.tolist()
print(categorical_columns)

0
[]


In [95]:
missing_values_table(dropped)

Your selected dataframe has 583 columns.
There are 530 columns that have missing values.


,Missing Values,% of Total Values,Data Type
516,1018,65.0,float64
518,1018,65.0,float64
382,1018,65.0,float64
246,1018,65.0,float64
245,1018,65.0,float64
...,...,...,...
458,1,0.1,float64
457,1,0.1,float64
456,1,0.1,float64
455,1,0.1,float64


In [96]:
#others then handle nullls

# import pandas as pd
# from sklearn.preprocessing import OneHotEncoder
# df=dropped
# # Select categorical columns
# categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
#
# # Create a copy of the original dataframe to preserve the original columns
# df_encoded = df.copy()
#
# # Create an empty DataFrame to store the encoded columns
# encoded_data_frames = []
#
# # Define a list of transformers for each categorical column
# for col in categorical_columns:
#     # Replace null values with "Others" in the original column
#     #df_encoded[col].fillna("Others", inplace=True)
#
#     # Create dummy variables using OneHotEncoder
#     encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
#     encoded_data = encoder.fit_transform(df_encoded[[col]])
#
#     # Get column names based on the original column and category
#     column_names = [category for category in encoder.get_feature_names_out([col])]
#
#     # Create a DataFrame for the encoded data
#     encoded_df = pd.DataFrame(encoded_data, columns=column_names, index=df_encoded.index)
#
#     try:
#       new_column_name = f"{col}_{'Others'}"
#       # Drop the original column and the dummy variables for "Others"
#       encoded_df.drop([new_column_name], axis=1, inplace=True)
#       encoded_df.loc[df[col].isnull(), column_names] = pd.NA
#     except:
#       print('This column doesnt have nulls',col)
#
#     # Add the encoded DataFrame to the list
#     encoded_data_frames.append(encoded_df)
#
# # Concatenate the original DataFrame and the list of encoded DataFrames
# df_encoded = pd.concat([df_encoded] + encoded_data_frames, axis=1)
#
# # Drop the original categorical columns
# df_encoded.drop(categorical_columns, axis=1, inplace=True)
#
# # Now, the df_encoded DataFrame contains the changes made to categorical columns, with null values preserved.

In [97]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
df=dropped
# Select categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Create a copy of the original dataframe to preserve the original columns
df_encoded = df.copy()

# Create an empty DataFrame to store the encoded columns
encoded_data_frames = []

# Define a list of transformers for each categorical column
for col in categorical_columns:

    # Create dummy variables using OneHotEncoder
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(df_encoded[[col]])

    # Get column names based on the original column and category
    column_names = [category for category in encoder.get_feature_names_out([col])]

    # Create a DataFrame for the encoded data
    encoded_df = pd.DataFrame(encoded_data, columns=column_names, index=df_encoded.index)

    # Add the encoded DataFrame to the list
    encoded_data_frames.append(encoded_df)

# Concatenate the original DataFrame and the list of encoded DataFrames
df_encoded = pd.concat([df_encoded] + encoded_data_frames, axis=1)

# Drop the original categorical columns
df_encoded.drop(categorical_columns, axis=1, inplace=True)

# Now, the df_encoded DataFrame contains the changes made to categorical columns, with null values preserved.

In [98]:
missing_values_table(df)

Your selected dataframe has 583 columns.
There are 530 columns that have missing values.


,Missing Values,% of Total Values,Data Type
516,1018,65.0,float64
518,1018,65.0,float64
382,1018,65.0,float64
246,1018,65.0,float64
245,1018,65.0,float64
...,...,...,...
458,1,0.1,float64
457,1,0.1,float64
456,1,0.1,float64
455,1,0.1,float64


In [99]:
missing_values_table(df_encoded)

Your selected dataframe has 583 columns.
There are 530 columns that have missing values.


,Missing Values,% of Total Values,Data Type
516,1018,65.0,float64
518,1018,65.0,float64
382,1018,65.0,float64
246,1018,65.0,float64
245,1018,65.0,float64
...,...,...,...
458,1,0.1,float64
457,1,0.1,float64
456,1,0.1,float64
455,1,0.1,float64


In [100]:
df_encoded.to_csv(os.path.join('Final-Datasets', 'UCI-Seacom-Encoded.csv'),index=False)